# Seurat objects  
## Load required packages  


In [ ]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library("scales")
library(harmony)


## Load seurat object (ALL GENES)



In [ ]:
seurat_obj <- readRDS(file = './results/rds/Norm_all_samples.rds')

seurat_obj
table(seurat_obj$orig.ident)


In [ ]:
counts <- GetAssayData(seurat_obj, assay = "RNA")
print(length(rownames(counts)))



## Create seurat object (METABO)


In [ ]:
genes <- read.csv("all_metabo.tsv", sep = '\t', header = TRUE)
head(genes, 3)


In [ ]:
features <- rownames(seurat_obj@assays$RNA@counts)
# keep genes names seeen un genes$Gene_name
matched_features <- features[features %in% genes$Gene_name]

counts <- GetAssayData(seurat_obj, assay = "RNA")
counts <- counts[(which(rownames(counts) %in% matched_features)),]
metabo_obj <- subset(seurat_obj, features = rownames(counts))


In [ ]:
metabo_obj
table(metabo_obj$orig.ident)


In [ ]:
counts <- GetAssayData(metabo_obj, assay = "RNA")
print(length(rownames(counts)))



##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
pc = 60



rerun RNAransform after subsetting the object, especially if you intend to rerun PCA (i.e. subclustering)

# QC metrics  


In [ ]:
metadata <- metabo_obj@meta.data




## Nb of reads  


In [ ]:
VlnPlot(metabo_obj,features="nCount_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "nCount_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nCount_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()


In [ ]:
nCount_metabo <- as.data.frame(rowSums(metabo_obj[["nCount_RNA"]]))
nCount_metabo$Genes <- "metabo"
colnames(nCount_metabo)[1] <- "nCount"

nCount_all <- as.data.frame(rowSums(seurat_obj[["nCount_RNA"]]))
nCount_all$Genes <- "all"
colnames(nCount_all)[1] <- "nCount"

other <- nCount_all
other$Genes <- "others"
dim(other)
other <- other[!(rownames(other) %in% rownames(metabo_obj$RNA)),]

expr <- rbind(nCount_all, nCount_metabo, other)
head(expr)
ggplot(expr, aes(x=Genes,y=nCount)) + geom_boxplot()



## Nb of genes  


In [ ]:
VlnPlot(metabo_obj,features="nFeature_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "nFeature_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nFeature_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
    scale_x_log10()
    scale_x_log10()
    
  scale_x_log10()
    


In [ ]:
ggplot(metabo_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()


## Gene expression  
We can pick the first 100 cells and look at the distributions of their expression values.


In [ ]:
as.tibble(
  metabo_obj@assays$RNA@data[,1:100]
) %>%
  pivot_longer(
    cols=everything(),
    names_to="cell",
    values_to="expression"
  ) %>%
  ggplot(aes(x=expression, group=cell)) +
  geom_density() +
  coord_cartesian(ylim=c(0,0.6), xlim=c(0,3))


In [ ]:
expression_metabo <- as.data.frame(rowSums(metabo_obj@assays$RNA@data))
expression_metabo$Genes <- "metabo"
colnames(expression_metabo)[1] <- "expression"

expression_all <- as.data.frame(rowSums(seurat_obj@assays$RNA@data))
expression_all$Genes <- "all"
colnames(expression_all)[1] <- "expression"

other <- expression_all
other$Genes <- "others"
dim(other)
other <- other[!(rownames(other) %in% rownames(metabo_obj$RNA)),]

expr <- rbind(expression_all, expression_metabo, other)
head(expr)
ggplot(expr, aes(x=Genes,y=log(expression+1))) + geom_boxplot()


## MT genes  
### All genes


In [ ]:
VlnPlot(metabo_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(metabo_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()



### Metabo only


In [ ]:
metabo_obj[["percent.mt"]] <- PercentageFeatureSet(metabo_obj, pattern = "^MT-", assay = 'RNA')
VlnPlot(metabo_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(metabo_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()


## Ribosomal genes  
### All genes  


In [ ]:
VlnPlot(metabo_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(metabo_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



### Metabo only  


In [ ]:
PercentageFeatureSet(metabo_obj,pattern="^RP[LS]", assay = 'RNA') -> metabo_obj[["percent.ribosomal"]] 
VlnPlot(metabo_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(metabo_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



## Largest gene  (All genes)


In [ ]:
VlnPlot(metabo_obj, features = "percent.largest_gene", group.by = "orig.ident", pt.size = 0)
RidgePlot(metabo_obj, features = "percent.largest_gene", group.by = "orig.ident")


In [ ]:
ggplot(metabo_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## FeatureScatter  


In [ ]:
FeatureScatter(metabo_obj, feature1 = "nCount_RNA", feature2 = "percent.mt", group.by ="orig.ident")

FeatureScatter(metabo_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by ="orig.ident") + geom_smooth(method = 'lm')



## Cell cycle scoring  


In [ ]:
metabo_obj@meta.data %>%
  group_by(orig.ident,Phase) %>%
  count() %>%
  group_by(orig.ident) %>%
  mutate(percent=100*n/sum(n)) %>%
  ungroup() %>%
  ggplot(aes(x=orig.ident,y=percent, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' )) +
  geom_col() +
  ggtitle("Percentage of cell cycle phases per sample")

as_tibble(metabo_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' ))+ geom_bar()


as_tibble(metabo_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' )) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))


# Identify high variable features  
*Included in the `RNAransform` function. It was done with all genes*


In [ ]:
metabo_obj <- FindVariableFeatures(metabo_obj, selection.method = "vst", nfeatures = 1000)

# identify the 10 most highly variable genes
top10 <- head(VariableFeatures(metabo_obj), 10)
top10

# plot the variable features 
plot <- VariableFeaturePlot(metabo_obj)
LabelPoints(plot = plot, points = head(top10,10), repel = TRUE)


We can plot out a graph of the variance vs mean and highlight the selected genes so we can see whether we think we’re likely to capture what we need.    



In [ ]:
as_tibble(HVFInfo(metabo_obj),rownames = "Gene") -> variance.data

variance.data %>% 
  mutate(hypervariable=Gene %in% VariableFeatures(metabo_obj)
) -> variance.data

variance.data %>% 
  ggplot(aes(log(mean),log(variance),color=hypervariable)) + 
  geom_point() + 
  scale_color_manual(values=c("black","red"))


In [ ]:
# identify the 10 most highly variable genes
top10 <- head(VariableFeatures(metabo_obj), 10)

# plot the variable features 
plot <- VariableFeaturePlot(metabo_obj)
LabelPoints(plot = plot, points = head(top10,10), repel = TRUE)


# Linear dimensional reduction 
## PCA  


In [ ]:
metabo_obj <- RunPCA(metabo_obj, verbose=FALSE)
VizDimLoadings(metabo_obj, dims = 1:2, reduction = "pca")


In [ ]:
mat <- Seurat::GetAssayData(metabo_obj, assay = "RNA", slot = "scale.data")
pca <- metabo_obj[["pca"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))
eigValues = (pca@stdev)^2  ## EigenValues

varExplained = eigValues / total_variance

pc1 = percent(varExplained[1], accuracy = 0.01)
pc2 = percent(varExplained[2], accuracy = 0.01)


In [ ]:
DimPlot(metabo_obj, reduction = "pca", group.by = "orig.ident") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))
DimPlot(metabo_obj, reduction = "pca", group.by = "line") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))
DimPlot(metabo_obj, reduction = "pca", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue',S='tomato3'))

DimPlot(metabo_obj, reduction = "pca", group.by = "orig.ident", label.size = 4, label = TRUE)
DimPlot(metabo_obj, reduction = "pca", group.by = "line", label.size = 4, label = TRUE)



### Split by condition


In [ ]:
DimPlot(metabo_obj, reduction = "pca", group.by="orig.ident", split.by = "condition")
DimPlot(metabo_obj, reduction = "pca", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(metabo_obj, reduction = "pca", group.by="orig.ident", split.by = "condition")+ NoLegend()



### Split by line


In [ ]:
DimPlot(metabo_obj, reduction = "pca", group.by="line", split.by = "condition")
DimPlot(metabo_obj, reduction = "pca", group.by = "line", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(metabo_obj, reduction = "pca", group.by="line", split.by = "condition")+ NoLegend()


# Harmony  
By default, Harmony accepts a normalized gene expression matrix and performs PCA.
*do_pca=FALSE if PCA already done*


In [ ]:
metabo_obj <- RunHarmony(metabo_obj, group.by.vars =c("line", "condition"), reduction = "pca", plot_convergence = TRUE)




## plots  


In [ ]:
mat <- Seurat::GetAssayData(metabo_obj, assay = "RNA", slot = "scale.data")
harmony <- metabo_obj[["harmony"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))
eigValues = (harmony@stdev)^2  ## EigenValues

varExplained = eigValues / total_variance

pc1 = percent(varExplained[1], accuracy = 0.01)
pc2 = percent(varExplained[2], accuracy = 0.01)

DimPlot(metabo_obj, reduction = "harmony", group.by = "orig.ident") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))


In [ ]:
DimPlot(metabo_obj, reduction = "harmony", group.by = "orig.ident")
DimPlot(metabo_obj, reduction = "harmony", group.by = "line")
DimPlot(metabo_obj, reduction = "harmony", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue',S='tomato3'))

DimPlot(metabo_obj, reduction = "harmony", group.by = "orig.ident", label.size = 4, label = TRUE)
DimPlot(metabo_obj, reduction = "harmony", group.by = "line", label.size = 4, label = TRUE)



### Split by condition


In [ ]:
DimPlot(metabo_obj, reduction = "harmony", group.by="orig.ident", split.by = "condition")
DimPlot(metabo_obj, reduction = "harmony", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(metabo_obj, reduction = "harmony", group.by="orig.ident", split.by = "condition")+ NoLegend()



### Split by line


In [ ]:
DimPlot(metabo_obj, reduction = "harmony", group.by="line", split.by = "condition")
DimPlot(metabo_obj, reduction = "harmony", group.by = "line", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(metabo_obj, reduction = "harmony", group.by="line", split.by = "condition")+ NoLegend()



### Save seurat object (metabo)


In [ ]:
saveRDS(metabo_obj, file = "./results/rds/all_samples_metabo.rds")



# Determine the dimensionality of the dataset  
how many components should we choose to include?  

**JackStrawPlot**  
*significant* PCs as those who have a strong enrichment of **low p-value**

* dashed line : uniform distribution
* solid curve above dash line : strong enrichments of features with low p-values

**ElbowPlot**  
ranking of principle components based on the percentage of variance explained by each one 


In [ ]:
ElbowPlot(metabo_obj, ndims = pc)

